<a href="https://colab.research.google.com/github/kooxc/Jupyter-Notebook/blob/main/langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 一、安装

In [ ]:
# 查看当前kernel下已安装的包  list packages
!pip list --format=columns

In [ ]:
# 安装 langchain 包
!pip install langchain
!pip install langchain-core
!pip install langchain-community
!pip install langchain-openai


In [ ]:
# 安装阿里通义千问的包
!pip install dashscope

### 二、基本调用示例

In [5]:
import os
from langchain_community.llms import Tongyi
from google.colab import userdata

os.environ["DASHSCOPE_API_KEY"] = userdata.get('QWEN_KEY')

llm = Tongyi()
llm.invoke("明源云是做什么业务的? 企业的CEO是谁?")

'明源云（MingYuan Cloud）是一家中国领先的房地产生态链数字化服务商，专注于为房地产开发商及产业链其他参与者提供企业级ERP产品和SaaS服务。其业务主要涵盖以下几个方面：\n\n1. **房地产开发领域的数字化解决方案**：提供覆盖房地产全生命周期的管理软件和服务，包括项目管理、成本管理、采购管理、营销管理等。\n2. **SaaS服务平台**：通过云端服务帮助房企实现线上化、智能化运营，提升效率。\n3. **大数据与人工智能应用**：利用大数据分析和AI技术助力房地产企业进行精准决策。\n\n关于企业CEO的信息，明源云的创始人兼董事长是高宇，而当前负责公司日常运营管理的首席执行官（CEO）是姜海洋。姜海洋在科技行业有着丰富的经验，并带领团队不断推动明源云的技术创新与发展。\n\n请注意，具体高管职位可能会随时间有所调整，建议查阅最新的官方公告以获取最准确的信息。'

### 三、聊天模型

#### 1、格式化输出

In [6]:
from langchain_community.chat_models.tongyi import ChatTongyi
from typing import Optional
from pydantic import BaseModel, Field

# 定义Pydantic模型
class Company(BaseModel):
    """公司简介信息"""

    domain: str = Field(description="官方域名地址")
    business: str = Field(description="主营业务")
    boss: str = Field(description="老板")
    income: str = Field(description="最近市场披露的收入")


llm = ChatTongyi(model="qwen-turbo")
structured_llm = llm.with_structured_output(Company)
structured_llm.invoke("介绍一下明源云这家公司")

Company(domain='www.mingyuanyun.com', business='房地产行业数字化转型解决方案提供商', boss='高宇', income='2022年收入超过10亿元')

#### 2、工具模式

In [7]:
# 函数名、类型和文档注释都是该工具的一部分
# 传递给模型的模式，定义良好的描述性模式，是提示工程的扩展，是使模型性能良好的重要组成部分
def add(a: int, b: int) -> int:
    """Add two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a + b


def multiply(a: int, b: int) -> int:
    """Multiply two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a * b

tools = [add, multiply]

In [8]:
llm = ChatTongyi(model="qwen-turbo")

llm_with_tools = llm.bind_tools(tools)

query = "3 * 12 = ？"

llm_with_tools.invoke(query)

AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'name': 'multiply', 'arguments': '{"a": 3, "b": 12}'}, 'index': 0, 'id': 'call_cb1da37be324418da22843', 'type': 'function'}]}, response_metadata={'model_name': 'qwen-turbo', 'finish_reason': 'tool_calls', 'request_id': 'e9d40412-99f3-9dc1-8b5c-5c82d671c509', 'token_usage': {'input_tokens': 263, 'output_tokens': 23, 'total_tokens': 286}}, id='run-8bb13ced-f797-42e8-803a-c1fb9566bedc-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_cb1da37be324418da22843', 'type': 'tool_call'}])

#### 3、缓存
在 LangChain 中，set_llm_cache 是一个用于设置缓存机制的函数。它的主要作用是通过缓存来减少对语言模型（LLM）的重复调用，从而节省时间和计算资源。以下是它的具体作用和使用场景：

**缓存的作用**

减少重复调用：如果相同的输入多次发送到 LLM，缓存可以存储之前的结果，避免重复调用模型，从而提高效率。
节省成本和时间：LLM 的调用通常需要消耗计算资源和时间，缓存可以显著减少这些开销。
提高性能：通过缓存，程序可以更快地获取结果，尤其是在需要频繁调用 LLM 的场景中。

**如何使用 set_llm_cache**

set_llm_cache 是 LangChain 提供的一个全局设置函数，用于启用缓存机制。它通常与 LangChain 的缓存实现（如 InMemoryCache 或 SQLiteCache）一起使用。

In [9]:
%%time
from langchain_core.caches import InMemoryCache
from langchain_core.globals import set_llm_cache

llm = ChatTongyi(model="qwen-turbo")

set_llm_cache(InMemoryCache())
llm.invoke("50个字以内介绍一下明源云这家公司")

CPU times: user 17.7 ms, sys: 1.07 ms, total: 18.8 ms
Wall time: 1.48 s


AIMessage(content='明源云是中国领先的房地产数字化解决方案提供商，专注于为房地产开发商及产业链相关企业提供软件云服务。', additional_kwargs={}, response_metadata={'model_name': 'qwen-turbo', 'finish_reason': 'stop', 'request_id': 'e0a71343-e7ec-9e41-82a6-bf539e13afe7', 'token_usage': {'input_tokens': 18, 'output_tokens': 22, 'total_tokens': 40}}, id='run-dfd684f4-3c77-4eaf-b49f-3fee75bfd7af-0')

In [10]:
%%time
#再次调用

llm.invoke("50个字以内介绍一下明源云这家公司")

CPU times: user 570 µs, sys: 0 ns, total: 570 µs
Wall time: 561 µs


AIMessage(content='明源云是中国领先的房地产数字化解决方案提供商，专注于为房地产开发商及产业链相关企业提供软件云服务。', additional_kwargs={}, response_metadata={'model_name': 'qwen-turbo', 'finish_reason': 'stop', 'request_id': 'e0a71343-e7ec-9e41-82a6-bf539e13afe7', 'token_usage': {'input_tokens': 18, 'output_tokens': 22, 'total_tokens': 40}}, id='run-dfd684f4-3c77-4eaf-b49f-3fee75bfd7af-0')